In [1]:
# !pip3 install --upgrade pandas
# !pip3 install --upgrade numpy
# !pip3 install --upgrade httplib2
# !pip3 install --upgrade instabot
# !pip3 install --upgrade google-api-python-client
# !pip3 install --upgrade vk-api

# TODO: add insta auth in a normal way

import json
from datetime import datetime, timedelta
import shutil

import pandas as pd
import numpy as np

from instabot import Bot

from BT_library import GoogleSheets, DF_works, Vkontakte, Insta

from secrets import INST_USERNAME, INST_PASSWORD, sobaka_spreadsheet_id

In [2]:
shutil.rmtree('config', ignore_errors=True)

In [3]:
CREDSTOSERVICE = 'credentials.json'
range_name_insta = 'Insta!A3:H'

In [4]:
d = DF_works()
bot = Bot()
ins = Insta()
g = GoogleSheets()

2021-06-14 18:34:45,193 - INFO - Instabot version: 0.117.0 Started


In [5]:
service = g.auth(CREDSTOSERVICE)

In [6]:
bot.login(username = INST_USERNAME,  password = INST_PASSWORD)
# # IN CASE OF LOGIN TROUBLES "ds_user" -- DELETE 'config' FOLDER NEARBY

user_id = bot.get_user_id_from_username("sobaka_ru")

user_media_list = bot.get_user_medias(user_id, 
                                      filtration=False, 
                                      is_comment=False) # list of followers

insta_values = ins.inst_get_posts(user_media_list, bot)

posts_from_sheets = g.get_values_as_df(service, 
                                       range_name_insta, 
                                       sobaka_spreadsheet_id)

df1 = d.inst_merge_dfs_max_likes(posts_from_sheets, insta_values)
df2 = d.inst_merge_dfs_text_cols(posts_from_sheets, insta_values)

result = d.inst_final_merge(df1, df2)

result.Date = result.Date.astype(str) # ?
result = result.values.tolist()

g.put_values(service, range_name_insta, sobaka_spreadsheet_id, result)

2021-06-14 18:34:45,221 - INFO - Not yet logged in starting: PRE-LOGIN FLOW!
2021-06-14 18:34:48,016 - INFO - Logged-in successfully as 'alexander1337n'!
2021-06-14 18:34:48,017 - INFO - LOGIN FLOW! Just logged-in: True
C:\Users\User\Desktop\Projects\BT_vk\vk\BT_library.py:367: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['Count_of_comments'] = df1['Count_of_comments'].astype('int32').fillna(0)
C:\Users\User\Desktop\Projects\BT_vk\vk\BT_library.py:368: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[

Wall time: 24.7 s


In [7]:
shutil.rmtree('config', ignore_errors=True)